# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.ai/seldon-core-1/tutorials/notebooks/seldon-core-setup#setup-cluster) with [Ambassador Ingress](https://docs.seldon.ai/seldon-core-1/tutorials/notebooks/seldon-core-setup#ambassador).

In [ ]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [14]:
VERSION = !cat ../version.txt
VERSION = VERSION[0]
VERSION

'1.19.0-dev'

## Serve Single Model

In [15]:

!helm upgrade -i mymodel ../helm-charts/seldon-single-model --set model.image=seldonio/mock_classifier:$VERSION --namespace seldon

Release "mymodel" has been upgraded. Happy Helming!
NAME: mymodel
LAST DEPLOYED: Thu Dec  4 12:11:31 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 3
TEST SUITE: None


In [16]:
!helm template mymodel ../helm-charts/seldon-single-model --set model.image=seldonio/mock_classifier:$VERSION | pygmentize -l json

---
# Source: seldon-single-model/templates/seldondeployment.json
{
  "kind": "SeldonDeployment",
  "apiVersion": "machinelearning.seldon.io/v1",
  "metadata": {
    "name": "mymodel",
    "namespace": "seldon",
    "labels": {}
  },
  "spec": {
      "name": "mymodel",
      "protocol": "seldon",
    "annotations": {},
    "predictors": [
      {
        "name": "default",
        "graph": {
          "name": "model",
          "type": "MODEL",
        },
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "name": "model",
                  "image": "seldonio/mock_classifier:1.19.0-dev",
                  "env": [
                      {
                        "name": "LOG_LEVEL",
                        "value": "INFO"
                      },
                    ],
                  "resources": {"requests":{"memory":"1Mi"}},
                }
              ]
            },
          }
        ],
        "r

In [17]:
!kubectl wait sdep/mymodel \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/mymodel condition met


### Get predictions

In [18]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymodel",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

#### REST Request

In [19]:
from tenacity import retry, stop_after_delay, wait_exponential

@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    r = sc.predict(transport="rest")
    assert r.success == True
    return r

predict()

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.71543328677795837
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.09963978586361734]}}, 'meta': {'requestPath': {'model': 'seldonio/mock_classifier:1.19.0-dev'}}}

#### GRPC Request

In [20]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    r = sc.predict(transport="grpc")
    assert r.success == True
    return r

predict()

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.555819139294561]}}}
Response:
{'meta': {'requestPath': {'model': 'seldonio/mock_classifier:1.19.0-dev'}}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.08620740652415673]}}}

In [21]:
!helm delete mymodel --namespace seldon

release "mymodel" uninstalled


## Serve REST AB Test

In [22]:
!helm upgrade -i myabtest ../helm-charts/seldon-abtest --namespace seldon

Release "myabtest" does not exist. Installing it now.
NAME: myabtest
LAST DEPLOYED: Thu Dec  4 12:12:21 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [23]:
!helm template ../helm-charts/seldon-abtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_2pods.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "release-name"
    },
    "spec": {
	"name": "release-name",
	"predictors": [
	    {
		"name": "default",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
                                "image": "seldonio/mock_classifier:1.19.0-dev",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
				{
                                "image": "seldonio/mock_classifier:1.19.0-dev",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
	

In [24]:
!kubectl wait sdep/myabtest \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/myabtest condition met


### Get predictions

In [25]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="myabtest",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

#### REST Request

In [26]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    r = sc.predict(transport="rest")
    assert r.success == True
    return r

predict()

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.26095295840328658
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.0656377202541611]}}, 'meta': {'requestPath': {'classifier-2': 'seldonio/mock_classifier:1.19.0-dev'}}}

#### gRPC Request

In [27]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    r = sc.predict(transport="grpc")
    assert r.success == True
    return r

predict()

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.163692647815478]}}}
Response:
{'meta': {'requestPath': {'classifier-2': 'seldonio/mock_classifier:1.19.0-dev'}}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.05991890833984506]}}}

In [28]:
!helm delete myabtest --namespace seldon

release "myabtest" uninstalled


## Serve REST Multi-Armed Bandit

In [29]:
!helm upgrade -i mymab ../helm-charts/seldon-mab --namespace seldon

Release "mymab" does not exist. Installing it now.
NAME: mymab
LAST DEPLOYED: Thu Dec  4 12:13:04 2025
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [30]:
!helm template ../helm-charts/seldon-mab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
		"labels": {"app":"seldon"},
		"name": "release-name"
    },
    "spec": {
	"name": "release-name",
	"predictors": [
	    {
		"name": "default",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
                                "image": "seldonio/mock_classifier:1.19.0-dev",				
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
			"spec":{
			    "containers":[
				{
                                "image": "seldonio/mock_classifier:1.19.0-dev",								    
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriod

In [31]:
!kubectl wait sdep/mymab \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/mymab condition met


### Get predictions

In [32]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(
    deployment_name="mymab",
    namespace="seldon",
    gateway_endpoint="localhost:8003",
    gateway="ambassador",
)

#### REST Request

In [33]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    r = sc.predict(transport="rest")
    assert r.success == True
    return r

predict()

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.13298918130078008
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.058212613572549546]}}, 'meta': {'requestPath': {'classifier-1': 'seldonio/mock_classifier:1.19.0-dev'}}}

#### gRPC Request

In [34]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def predict():
    r = sc.predict(transport="grpc")
    assert r.success == True
    return r

predict()

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 1], 'values': [0.1279822023238696]}}}
Response:
{'meta': {'requestPath': {'classifier-1': 'seldonio/mock_classifier:1.19.0-dev'}}, 'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.05793871786684459]}}}

In [35]:
!helm delete mymab --namespace seldon

release "mymab" uninstalled
